<a href="https://colab.research.google.com/github/QoutiOussama13/AI-EarthHack-MOAA-/blob/main/Idea_Evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Some notes :

*   Ensure the you are using google collab to run this notebook with Hardware accelertor set as **GPU**

*   Some cells may take some time to run if you have **Colab Pro** that would fix the problem

*   make sure to replace the ```./path_to_data``` with the real path of the data folder *preferebly on Google drive*

* Enjoy 😀!





# **requirements**

In [ ]:
!pip install langchain sentence-transformers chromadb

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

# **Improting the libreries**

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from huggingface_hub import InferenceClient
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA,ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp
import time
from langchain.prompts import PromptTemplate

this is to connect with google drive

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
loader = DirectoryLoader('./path_to_data', glob="**/*.txt", loader_cls=TextLoader)
pages = loader.load()

In [45]:
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=150,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

In [46]:
chunks_text = split_text(pages)

Split 10 documents into 372 chunks.
Does this mean that the whole organisation sells the same thing the same way, year after year? No - solution repeatability also involves the ability to sense, react and adapt to changing market conditions.
Look at the list on the right. Where would you place your organisation?
    REPEATABILITY OF SOLUTIONS
  UNSCALABLE
    1: Random
Your solutions are created from scratch to address unique customer needs, with little standardisation. Every sale seems hand-crafted.
    2: Common Components
You have created a family of common components from which you build solutions, but sales still often have somewhat unique elements to them.
    3: Standardised
You have established standardised product or service offerings that you are marketing & selling in a reasonably consistent fashion.
    4: Replicable
The majority of your revenues come from clearly defined, standardised product and service offerings sold in a repeatable, consistent way.
    5: Highly Repeata

The embedding model
[e5-small-v](https://huggingface.co/intfloat/e5-small-v2)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name='intfloat/e5-small-v2')

saving the data we have in the vector database [ChromaDB](https://https://www.trychroma.com)

In [47]:
db = Chroma.from_documents(chunks_text, embedding=embedding)

initializing the [LLM](https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GGUF) we are going to work with  

In [11]:
MODEL_ID = "TheBloke/Mistral-7B-OpenOrca-GGUF"
MODEL_BASENAME = "mistral-7b-openorca.Q4_K_M.gguf"

In [12]:
model_path = hf_hub_download(
            repo_id=MODEL_ID,
            filename=MODEL_BASENAME,
            resume_download=True,
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-openorca.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [48]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

CONTEXT_WINDOW_SIZE = 1900
MAX_NEW_TOKENS = 2500
N_BATCH = 1
n_gpu_layers = 40
kwargs = {
          "model_path": model_path,
          "n_ctx": CONTEXT_WINDOW_SIZE,
          "max_tokens": MAX_NEW_TOKENS,
          "n_batch": N_BATCH,
          "n_gpu_layers": n_gpu_layers,
          "callback_manager": callback_manager,
          "verbose":True,
      }

Instance creation using [LlamaCpp](https://github.com/ggerganov/llama.cpp)

In [49]:
llm = LlamaCpp(**kwargs)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Initializing some examples to see the performance of the model

In [65]:
test = "hey how are you?"
business_idea = """
My proposal is to establish an intuitive, user-friendly digital sharing platform named 'WasteNot'. This platform will revolutionize waste management by serving as a bridge that connects waste-generating companies with businesses seeking these materials for reuse or recycling. It supports the circular economy by extending the life of materials and reducing the need for fresh raw material extraction. Furthermore, it benefits businesses financially by means of lowered waste disposal costs for waste generators and reduced raw material procurement expenses for recycling companies. The platform is easy to implement through web and mobile-based applications, and it is scalable to cater to businesses of varying sizes across different industries, thus promoting a more sustainable global business environment.
"""

In [51]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    verbose=False,
)

Creating a template for the model to ensure it responds according to our specifications.

In [81]:
new_template = """
You are and expert idea validator in terms of circular economy principles to address climate change.
Carefully evaluate the provided business idea based on six key metrics— circular economy (which include resource efficiency, waste management, long term viability, closed loop system, life cycle analysis), feasibility, scalability, innovation, and social impact and generate a score (0 to 10) for each metric, avoiding assumptions.
Ensure the evaluation is grounded in the context's content, avoiding assumptions.
Provide a comprehensive score and insights for each metric to guide investors in aligning their ambitions with the most promising business opportunities.
Add at the end the overall score for the evalution .
remember you don't read any buisness idea from the context , your only interest is the buisness idea provided by the user
If the user provided something other than a business idea  , just say that you cannot answer politly in just few words , don't try to make up an answer from something in the context.
Context: {context}
Business Idea: {question}
your answer with the evaluation and the short description for each metric and ways of improving the business idea :  """


NEW_PROMPT = PromptTemplate(template=new_template, input_variables=['context', 'question'])

In [82]:
qa.combine_documents_chain.llm_chain.prompt = NEW_PROMPT

## running the tests:

*Note :the results may change even using the same prompt , we suggest that experiment with different prompts multiple times*

first test

In [84]:
res_1 = qa(business_idea)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit



1. Circular Economy: 8/10 (Resource efficiency, waste management, long term viability, closed loop system, life cycle analysis)
The business idea presents a model that leverages the principles of microscale mechanics to create energy-efficient cooling systems. It promotes resource efficiency and waste reduction by extending the life of chillers, reducing energy consumption, and minimizing environmental impacts. A closed-loop system is created through the minimal electrical requirements, and life cycle analysis can identify areas for improvement. However, it could be improved by further research on materials used in chillers and exploring opportunities for recycling or repurposing these materials at the end of their life cycles.

2. Feasibility: 8/10
The business idea appears to be feasible, as it relies on proven scientific principles and existing technologies. However, further research and development are needed to refine the model and ensure its scalability. This includes optimizing

In [8]:
print("Business Idea :")
print (business_idea)
print("LLM response :")
print(res_1['result'])

 
Business Idea :
My proposal is to establish an intuitive, user-friendly digital sharing platform named 'WasteNot'. 
This platform will revolutionize waste management by serving as a bridge that connects waste-generating companies with businesses seeking these materials for reuse or recycling.
It supports the circular economy by extending the life of materials and reducing the need for fresh raw material extraction.
Furthermore, it benefits businesses financially by means of lowered waste disposal costs for waste generators and reduced raw material procurement expenses for recycling companies. 
The platform is easy to implement through web and mobile-based applications, and it is scalable to cater to businesses of varying sizes across different industries, thus promoting a more sustainable global business environment.

LLM response : 
1. Circular Economy: 8/10 (Resource efficiency, waste management, long term viability, closed loop system, life cycle analysis)
The business idea presen

second test

In [79]:
res_2 = qa(test)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit



Your buisness idea is not provided. Please provide a business idea for me to evaluate. If you need an evaluation, please ask in the context.

In [9]:
print("testing to see if it will responde to a random prompt :")
print (test)
print("LLM response :")
print(res_2['result'])

testing to see if it will responde to a random prompt :
hey how are you?
LLM response :
Your buisness idea is not provided. Please provide a business idea for me to evaluate. If you need an evaluation, please ask in the context.


if you have any question/issue with running the code please feel free to reach out to [Oussama](https://https://www.linkedin.com/in/oussama-qouti-105bb820a/)